# Import Libraries

In [58]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import calendar

# Initialize the DataFrame

In [59]:
df = pd.read_csv("Data/RefinedFile.csv", na_values='-', index_col=0)
# df

# Generating an image of your Heatmap

In [60]:
def heat(df:pd.DataFrame, normalize, params, annot=False):

    ## Create proper fomrmat
    if params['Title'] == "Hour":
        df = df.pivot_table(index='Crm Cd Desc', columns=df['TIME OCC'] // 100, values='Count', aggfunc='sum', fill_value=0)

    elif params['Title'] == "Light":
        df = df.pivot_table(index='Crm Cd Desc', columns='Sun', values='Count', aggfunc='sum', fill_value=0)

    elif params['Title'] == "Weekday":
        ## Convert str to datetime
        df['DATE OCC'] = pd.to_datetime(df['DATE OCC'], format='%m/%d/%Y')

        ## Create Weekday Column
        df['Weekday'] = df['DATE OCC'].dt.strftime('%A')

        pivoted_table = df.pivot_table(index='Crm Cd Desc', columns='Weekday', values='Count', aggfunc='sum', fill_value=0)

        ## Sort Weekday's order
        weekday_order = list(calendar.day_name)
        df = pivoted_table.reindex(columns=weekday_order)

    else:
        print("Please enter a valid title (Hour, Light, Weekday)")


    ## Normalizing
    if normalize:
        min_val = df.values.min()
        max_val = df.values.max()
        df = (df - min_val) / (max_val - min_val)
        
    ## Don't print heatmap in notebook
    plt.ioff()
    plt.figure(figsize=(16,8))
    plt.title(f'Crimes in a given Time frame')
    plt.subplots_adjust(left=0.4)
    
    heatmap = sns.heatmap(df, annot=annot)
    heatmap.set_ylabel('Crimes')
    heatmap.set_xlabel(params['Xlabel'])
    fig = heatmap.get_figure()
    fig.savefig(f"figures/heatmaps/{params['Title']}/{params['Title'][0]}({params['Start']}-{params['End']}).png") 




def generate_heatmaps(df:pd.DataFrame, start, end, normalize=True):
    ## Only keep values in range
    value_counts = df["Crm Cd Desc"].value_counts()
    selected_values = value_counts[value_counts.between(start, end)].index.tolist()
    df = df[df["Crm Cd Desc"].isin(selected_values)]

    ## Weekday Heatmap
    weekday_group = df.groupby(["Crm Cd Desc", "DATE OCC"]).size().reset_index(name='Count')
    heat(weekday_group, normalize, {"Title": "Weekday", "Start": start, "End": end, "Xlabel": "Day of the Week"})

    ## Light Heatmap
    light_group = df.groupby(["Crm Cd Desc", "Sun"]).size().reset_index(name='Count')
    heat(light_group, normalize, {"Title": "Light", "Start": start, "End": end, "Xlabel": "Daylight"})

    ## Hour Heatmap
    hour_group = df.groupby(["Crm Cd Desc", "TIME OCC"]).size().reset_index(name='Count')
    heat(hour_group, normalize, {"Title": "Hour", "Start": start, "End": end, "Xlabel": "Time (0-23h)"})


# Generation calls

In [63]:
# generate_heatmap(df, from, to, normalize)

if __name__ == "__main__":
    generate_heatmaps(df, 0, 1000, True)
    generate_heatmaps(df, 1000, 5000, True)

    for i in range(10000, 70000, 10000):
        generate_heatmaps(df, i, i+10000, True)